In [1]:
# coding: utf-8
"""
ノード分割アルゴリズム部分のコードの書き換え
alpha = 全単語数に対する語彙の頻度数, beta = 頻度の割合
グリッドサーチによるパラメータの調査
"""
from igraph import *
import csv
import collections
import pickle
import numpy as np
from openopt import QP
import copy
import re

# csvファイルの読み込み
def readcsv(path):
    f = open(path, "rb")
    dataReader = csv.reader(f)
    arr = [row for row in dataReader]
    return arr

def writecsv(arr, path):
    f = open(path, "ab")
    dataWriter = csv.writer(f)
    dataWriter.writerows(arr)
    f.close()

def readdump(path):
    f = open(path, "r")
    arr = pickle.load(f)
    f.close()
    return arr

# 有向エッジリストを入力して、重み付き無向ネットワークを出力する
def cal_edgelist_to_network(list_edge):
    # 有向グラフならば
    if len(list_edge[0]) == 2:
        # 有向エッジリストを無向エッジリストに変換する
        list_edge = [tuple(sorted(row)) for row in list_edge]
        # ノードリスト
        list_vertices = list(set([word for row in list_edge for word in row]))
        # エッジリストとそのweightを作成
        tuple_edge, tuple_weight = zip(*collections.Counter(list_edge).items())
        return {"vertex": list_vertices, "edge": list(tuple_edge), "weight": list(tuple_weight)}
    # 重み付き向こうグラフならば
    if len(list_edge[0]) == 3:
        list_vertex = []
        list_edge_rev = []
        list_weight = []
        for row in list_edge:
            list_vertex.append(row[0])
            list_vertex.append(row[1])
            list_edge_rev.append([row[0], row[1]])
            list_weight.append(float(row[2]))
        list_vertex = list(set(list_vertex))
        return {"vertex": list_vertex, "edge": list_edge_rev, "weight": list_weight}
    
    
# クラスタリング済みのネットワークを元にサブグラフのリストを作成
# vertexには全てのvertexを代入する（PageRankを計算するため）
def cal_cluster_to_network(dict_network):
    if dict_network.has_key("cluster") == False:
        print "クラスタリングができていません"
    
    # クラスタごとにwordをまとめる
    dict_cluster = collections.defaultdict(list)
    for word, cluster in zip(dict_network["vertex"], dict_network["cluster"]):
        dict_cluster[cluster].append(word)

    # リストに変換
    list_cluster_vertex = [row[1] for row in dict_cluster.items()]
    
    # 同様にエッジとウェイトのリストも作成する
    list_cluster_edge = []
    list_cluster_weight = []
    for cluster_vertex in list_cluster_vertex:
        list_cluster_edge_one = []
        list_cluster_weight_one = []
        # エッジリストの中に、一つでもノードが含まれていれば、そのクラスのノードに含める
        for row, weight in zip(dict_network["edge"], dict_network["weight"]):
            # and と or を切り替えることによって性能の比較
            if row[0] in cluster_vertex or row[1] in cluster_vertex:
                list_cluster_edge_one.append(row)
                list_cluster_weight_one.append(weight)
        list_cluster_edge.append(list_cluster_edge_one)
        list_cluster_weight.append(list_cluster_weight_one)
    
    # まとめる
    list_dict_network = [{"vertex": dict_network["vertex"],
                          "edge": cluster_edge,
                          "weight": cluster_weight}
                         for cluster_edge, cluster_weight
                         in zip(list_cluster_edge, list_cluster_weight)]
    
    return list_dict_network

# f_measureを計算する
def cal_f_measure(list_predict_measure):
    # 生成したクラスタ内のカウント
    dict_predict_cluster = collections.defaultdict(list)
    for row in list_predict_measure:
        dict_predict_cluster[row[0]].append(row[1])
        
    # もとあるクラス内のカウント
    dict_measure_cluster = collections.defaultdict(list)
    for row in list_predict_measure:
        dict_measure_cluster[row[1]].append(row[0])
    
    # local_purityの計算
    list_purity = []
    for row in dict_predict_cluster.items():
        major_class = sorted(collections.Counter(row[1]).items(), key=lambda x: x[1], reverse=True)[0][1]
        class_num = len(row[1])
        list_purity.append([major_class, class_num])
    purity = float(np.sum(zip(*list_purity)[0])) / np.sum(zip(*list_purity)[1])
    print "Purity: ", purity
    
    # inverse_purityの計算
    list_inverse_purity = []
    for row in dict_measure_cluster.items():
        major_class = sorted(collections.Counter(row[1]).items(), key=lambda x: x[1], reverse=True)[0][1]
        class_num = len(row[1])
        list_inverse_purity.append([major_class, class_num])
    inverse_purity = float(np.sum(zip(*list_inverse_purity)[0])) / np.sum(zip(*list_inverse_purity)[1])
    print "Inverse Purity: ", inverse_purity
    
    print "F-value: ", 2 / (1 / purity + 1 / inverse_purity)
    return purity, inverse_purity, 2 / (1 / purity + 1 / inverse_purity)
    
# 凸２次計画問題を解いてp(topic)を求めるための関数
def cal_prob_topic(dict_network_master, list_dict_network_sub):
    prob_master = np.array([row[1] for row in sorted(zip(dict_network_master["vertex"], dict_network_master["page_rank"]), key=lambda x: x[0])])
    
    for i, dict_network_sub in enumerate(list_dict_network_sub):
        if i == 0:
            prob_sub = np.array([row[1] for row in sorted(zip(dict_network_sub["vertex"], dict_network_sub["page_rank"]), key=lambda x: x[0])])
        else:
            list_tmp = np.array([row[1] for row in sorted(zip(dict_network_sub["vertex"], dict_network_sub["page_rank"]), key=lambda x: x[0])])
            prob_sub = np.vstack((prob_sub, list_tmp))
    
    H = 2 * prob_sub.dot(prob_sub.T)
    f = -2 * prob_master.dot(prob_sub.T)
    Aeq = np.ones(len(list_dict_network_sub))
    beq = 1
    lb = np.zeros(len(list_dict_network_sub))
    
    p = QP(H, f, Aeq=Aeq, beq=beq, lb=lb)
    r = p.solve("cvxopt_qp")
    k_opt = r.xf
    return k_opt

In [2]:
# クラスタリング後に、ノードごとにどのクラスタにエッジを持っているのか計算
def cal_cluster_to_edge(dict_network, low_fleq=0.05, low_rate=0.8, flag=0):
    if dict_network.has_key("cluster") == False:
        print "クラスタリングができていません"
    
    # wordをclusterに変換するための辞書を作成する
    dict_word_to_cluster = {}
    for word, cluster in zip(dict_network["vertex"], dict_network["cluster"]):
        dict_word_to_cluster[word] = cluster
        
    # wordをidに変換するための辞書を作成する
    dict_word_to_id = {}
    for i, word in enumerate(dict_network["vertex"]):
        dict_word_to_id[word] = i
    
    # clusterの数
    num_cluster = len(set(dict_network["cluster"]))
    
    # id_to_clusterのマトリックス
    matrix_id_to_cluster = np.zeros((len(dict_word_to_id), num_cluster))
    for row, weight in zip(dict_network["edge"], dict_network["weight"]):
        matrix_id_to_cluster[dict_word_to_id[row[0]]][dict_word_to_cluster[row[1]]] += weight
        matrix_id_to_cluster[dict_word_to_id[row[1]]][dict_word_to_cluster[row[0]]] += weight
        
    #総単語数を求める
    total_voc = np.sum(matrix_id_to_cluster)
    
    # 指定したハイパーパラメータよりも高い値を記録した単語を所属クラスタとともに辞書に登録する
    dict_word_to_list_cluster = {}
    for cluster, word, row in zip(dict_network["cluster"], dict_network["vertex"], matrix_id_to_cluster):
        top_num = max(row)
        if float(np.sum(row))/total_voc>=low_fleq and len(np.where(row/top_num>=low_rate)[0])>=2:
            #print word
            dict_word_to_list_cluster[word] = np.where(row/top_num>=low_rate)[0]
            
    # 分割する単語が存在するかしないかflagを立てる
    if len(dict_word_to_list_cluster) > 0:
        outflag = True
    else:
        outflag = False

    # 新しく分裂するノードクラスターを元のクラスターに記録
    for word, row in dict_word_to_list_cluster.items():
        for num in row:
            # dict_word_to_cluster[word+"_"+str(num)] = num
            dict_word_to_cluster[word+"_"+str(num)] = dict_word_to_cluster[word]
    
    # ここの計算が間違ってそう => diffsplitで確認したが大丈夫そう・・・
    for word in dict_word_to_list_cluster.keys():
        list_edge_new = []
        list_weight_new = []
        for row, weight in zip(dict_network["edge"], dict_network["weight"]):
            if row[0] == word:
                if dict_word_to_cluster[row[1]] in dict_word_to_list_cluster[word]:
                    list_edge_new.append([row[0]+"_"+str(dict_word_to_cluster[row[1]]), row[1]])
                    list_weight_new.append(weight)
                else:
                    for num in dict_word_to_list_cluster[word]:
                        # flagの値によって、weightの計算法を切り替え
                        if flag == 0:
                            weight_tmp = float(weight)/len(dict_word_to_list_cluster[word])
                            list_edge_new.append([row[0]+"_"+str(num), row[1]])
                            list_weight_new.append(weight_tmp)
                        else:
                            weight_tmp = weight/len(dict_word_to_list_cluster[word])
                            if weight_tmp > 0:
                                list_edge_new.append([row[0]+"_"+str(num), row[1]])
                                list_weight_new.append(weight_tmp)
                                
            elif row[1] == word:
                if dict_word_to_cluster[row[0]] in dict_word_to_list_cluster[word]:
                    list_edge_new.append([row[0], row[1]+"_"+str(dict_word_to_cluster[row[0]])])
                    list_weight_new.append(weight)
                else:
                    for num in dict_word_to_list_cluster[word]:
                        # flagの値によって、weightの計算法を切り替え
                        if flag == 0:
                            weight_tmp = float(weight)/len(dict_word_to_list_cluster[word])
                            list_edge_new.append([row[0], row[1]+"_"+str(num)])
                            list_weight_new.append(weight_tmp)
                        else:
                            weight_tmp = weight/len(dict_word_to_list_cluster[word])
                            if weight_tmp > 0:
                                list_edge_new.append([row[0], row[1]+"_"+str(num)])
                                list_weight_new.append(weight_tmp)
            else:
                list_edge_new.append([row[0], row[1]])
                list_weight_new.append(weight)
        else:
            dict_network["edge"] = copy.deepcopy(list_edge_new)
            dict_network["weight"] = copy.deepcopy(list_weight_new)
            list_vertices = list(set([word for row in dict_network["edge"] for word in row]))
            dict_network["vertex"] = list_vertices
    
    return dict_network, outflag

###メイン部分
1. 有向エッジリストから無向重み付きエッジリストを作成
2. louvain法によるクラスタリング
3. クラスタごとにpagerankを計算し、p(word|topic)とする
4. クラスタごとにp(word|topic)を出力する辞書を作成

In [3]:
def main(path, testpath, alpha, beta, n=0, p_k=True, DN=True): 
    # エッジリストの読み込み
    list_edge = readcsv(path)
    # 元のネットワークを作成する（無向）
    dict_network_master = cal_edgelist_to_network(list_edge)
    # g = Graph(directed=True)
    g_master = Graph()
    g_master.add_vertices(dict_network_master["vertex"])
    g_master.add_edges(dict_network_master["edge"])
    # 元のネットワークのpagerankを求める
    dict_network_master["page_rank"] = g_master.pagerank(directed=False, weights=dict_network_master["weight"])

    counter=0
    while DN:
        # louvain法によるクラスタリング、vertexと同じ長さのクラスタ番号が書かれたリストがreturn
        if n == 0:
            dict_network_master["cluster"] = g_master.community_fastgreedy(weights=dict_network_master["weight"]).as_clustering().membership
        else:
            dict_network_master["cluster"] = g_master.community_fastgreedy(weights=dict_network_master["weight"]).as_clustering(n=n).membership
        # クラスタリングの結果をもとに、どのノードから何本のエッジが出ているか計算する
        dict_network_master, DN = cal_cluster_to_edge(dict_network_master, low_fleq=alpha, low_rate=beta, flag=0)
        counter +=1
        #print counter, "回目"
        #print len(dict_network_master["weight"])
        #print len(dict_network_master["edge"])
        g_master = Graph()
        g_master.add_vertices(dict_network_master["vertex"])
        g_master.add_edges(dict_network_master["edge"])
    if n == 0:
        dict_network_master["cluster"] = g_master.community_fastgreedy(weights=dict_network_master["weight"]).as_clustering().membership
    else:
        dict_network_master["cluster"] = g_master.community_fastgreedy(weights=dict_network_master["weight"]).as_clustering(n=n).membership
    # 元のネットワークのpagerankを求める
    dict_network_master["page_rank"] = g_master.pagerank(directed=False, weights=dict_network_master["weight"])
    # クラスタ結果をもとにサブグラフのリストを作成
    list_dict_network_sub = cal_cluster_to_network(dict_network_master)
    # サブクラスタごとに中心性の計算
    for i, dict_network_sub in enumerate(list_dict_network_sub):
        g_sub = Graph()
        g_sub.add_vertices(dict_network_sub["vertex"])
        g_sub.add_edges(dict_network_sub["edge"])
        list_dict_network_sub[i]["page_rank"] = g_sub.pagerank(directed=False, weights=dict_network_sub["weight"])
    print "クラスタ数: ", len(list_dict_network_sub)

    # トピックごとにwordを入力したらp(word|topic)が出るような辞書を作成
    list_dict_prob = []
    for i in range(len(list_dict_network_sub)):
        list_word_page = sorted(zip(list_dict_network_sub[i]["vertex"], list_dict_network_sub[i]["page_rank"]), key=lambda x: x[1], reverse=True)
        list_word_page_rev = []
        for row in list_word_page:
            pattern = "_[0-9|_]+"
            word = re.sub(pattern, "", row[0])
            list_word_page_rev.append([word, row[1]]) 
        list_dict_prob.append({row[0]: row[1] for row in list_word_page_rev})
    
    # 凸２次計画問題を解いて、p(topic)を求める
    list_prob_topic = cal_prob_topic(dict_network_master, list_dict_network_sub)
    
    list_dict_words = dict_network_master["vertex"]
    list_sep_words = readdump(testpath)
    list_sep_words_rev = []
    for row in list_sep_words:
        list_sep_words_rev.append([row[2], row[1]])
    # 確率が最大になるクラスを予想する
    # 実質ラベルがintじゃない場合は、エラーとして、記録しない
    list_predict_measure = []
    list_words_rev = []
    error_count = 0
    for row in list_sep_words_rev:
        try:
            class_tmp = 0
            prob_tmp = 0
            predict_class = []
            for num, dict_prob in enumerate(list_dict_prob):
                prob_tmp_tmp = 1
                for word in row[0]:
                    prob_tmp_tmp *= float(dict_prob[word])
                if p_k:
                    prob_tmp_tmp *= list_prob_topic[num]
                if prob_tmp_tmp > prob_tmp:
                    class_tmp = num
                    prob_tmp = prob_tmp_tmp
            list_predict_measure.append([class_tmp, row[1]])
        except:
            error_count += 1
    #print "エラーデータ数: ", error_count
    print "テストデータ数: ",len(list_predict_measure)
    return cal_f_measure(list_predict_measure)

### クラスタ数指定なしで最適だと判断されたF-value

In [11]:
purity, inv, fvalue = main("./files/rakuten_corpus/rakuten_corpus_edgelist.csv", 
                           "./files/rakuten_corpus/annotation/all_sep.dump",
                           alpha=0.002, beta=0.6, n=0, p_k=True, DN=True)

クラスタ数:  10

------------------------- OpenOpt 0.5625 -------------------------
problem: unnamed   type: QP
solver: cvxopt_qp
     pcost       dcost       gap    pres   dres
 0: -2.1674e-03 -1.0038e+00  1e+00  8e-17  3e+00
 1: -2.1694e-03 -1.3776e-02  1e-02  5e-17  4e-02
 2: -2.2949e-03 -3.5358e-03  1e-03  7e-17  4e-03
 3: -2.5673e-03 -2.7433e-03  2e-04  5e-17  1e-18
 4: -2.6124e-03 -2.6346e-03  2e-05  9e-17  1e-18
 5: -2.6220e-03 -2.6251e-03  3e-06  8e-17  2e-18
 6: -2.6237e-03 -2.6240e-03  4e-07  7e-17  1e-18
 7: -2.6239e-03 -2.6240e-03  2e-08  5e-17  1e-18
 8: -2.6240e-03 -2.6240e-03  4e-10  1e-16  1e-18
Optimal solution found.
istop: 1000 (optimal)
Solver:   Time Elapsed = 0.0 	CPU Time Elapsed = 0.0
objFuncValue: -0.0026239608 (feasible, MaxResidual = 1.11022e-16)
テストデータ数:  1998
Purity:  0.595595595596
Inverse Purity:  0.679179179179
F-value:  0.634647211012


### bigramではなく、共起グラフを用いた場合

In [5]:
list_purity = []
list_inv = []
list_fvalue = []
list_alpha = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005,
              0.0006, 0.0007, 0.0008, 0.0009, 0.001]
list_beta = [float(num) / 10 for num in range(5, 10)]
for alpha in list_alpha:
    list_purity_tmp = []
    list_inv_tmp = []
    list_fvalue_tmp = []
    for beta in list_beta:
        print "alpha: ", alpha
        print "beta: ", beta
        purity, inv, fvalue = main("./files/rakuten_corpus/rakuten_corpus_edgelist_co.csv", 
                                   "./files/rakuten_corpus/annotation/all_sep.dump",
                                   alpha=alpha, beta=beta, n=0, p_k=True, DN=True)
        list_purity_tmp.append(purity)
        list_inv_tmp.append(inv)
        list_fvalue_tmp.append(fvalue)
    list_purity.append(list_purity_tmp)
    list_inv.append(list_inv_tmp)
    list_fvalue.append(list_fvalue_tmp)

writecsv(list_purity, "./files/purity_co_grid_4.csv")
writecsv(list_inv, "./files/inv_co_grid.csv_4")
writecsv(list_fvalue, "./files/fvalue_co_grid_4.csv")

alpha:  0.0001
beta:  0.5
クラスタ数:  8

------------------------- OpenOpt 0.5625 -------------------------
problem: unnamed   type: QP
solver: cvxopt_qp
     pcost       dcost       gap    pres   dres
 0: -1.3824e-03 -1.0034e+00  1e+00  3e-16  3e+00
 1: -1.3871e-03 -1.3408e-02  1e-02  8e-17  4e-02
 2: -1.6239e-03 -3.0780e-03  1e-03  6e-17  4e-03
 3: -1.9737e-03 -2.1992e-03  2e-04  9e-17  1e-18
 4: -2.0269e-03 -2.0503e-03  2e-05  1e-16  1e-18
 5: -2.0334e-03 -2.0362e-03  3e-06  1e-16  6e-19
 6: -2.0342e-03 -2.0345e-03  3e-07  1e-16  1e-18
 7: -2.0343e-03 -2.0343e-03  3e-08  1e-16  2e-18
 8: -2.0343e-03 -2.0343e-03  1e-09  1e-16  1e-18
Optimal solution found.
istop: 1000 (optimal)
Solver:   Time Elapsed = 0.0 	CPU Time Elapsed = 0.0
objFuncValue: -0.0020343042 (feasible, MaxResidual = 0)
テストデータ数:  1998
Purity:  0.617117117117
Inverse Purity:  0.728228228228
F-value:  0.668084379022
alpha:  0.0001
beta:  0.6
クラスタ数:  7

------------------------- OpenOpt 0.5625 -------------------------
proble

### 共起グラフ+半windowを使用した場合

In [27]:
for n in range(3, 21):
    purity, inv, fvalue = main("./files/rakuten_corpus/rakuten_corpus_edgelist_window_1_0.2.csv", 
             "./files/rakuten_corpus/annotation/all_sep.dump",
             alpha=0.002, beta=0.6, n=n)

1070
40315
40315
クラスタ数:  3

------------------------- OpenOpt 0.5625 -------------------------
problem: unnamed   type: QP
solver: cvxopt_qp
     pcost       dcost       gap    pres   dres
 0: -2.0724e-03 -1.0029e+00  1e+00  6e-17  2e+00
 1: -2.0731e-03 -1.2883e-02  1e-02  1e-16  2e-02
 2: -2.1188e-03 -2.7799e-03  7e-04  8e-17  1e-03
 3: -2.1729e-03 -2.2127e-03  4e-05  1e-16  3e-19
 4: -2.1732e-03 -2.1736e-03  4e-07  8e-17  7e-19
 5: -2.1732e-03 -2.1732e-03  4e-09  1e-16  5e-19
Optimal solution found.
istop: 1000 (optimal)
Solver:   Time Elapsed = 0.0 	CPU Time Elapsed = 0.0
objFuncValue: -0.0021731568 (feasible, MaxResidual = 0)
テストデータ数:  1998
Purity:  0.430930930931
Inverse Purity:  0.741241241241
F-value:  0.545011706839
1070
40315
40315
クラスタ数:  4

------------------------- OpenOpt 0.5625 -------------------------
problem: unnamed   type: QP
solver: cvxopt_qp
     pcost       dcost       gap    pres   dres
 0: -2.3152e-03 -1.0027e+00  1e+00  6e-17  3e+00
 1: -2.3155e-03 -1.2703e-02 

In [4]:
list_alpha = [float(num) / 100 for num in range(0, 11)]
list_beta = [float(num) / 10 for num in range(0, 11)]

In [6]:
list_beta

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]